# Segmentation Performance
Evaluation of membrane detection using watershed/variation of information (VI) and pixel-wise membrane error. Variation of Information is calculated using [partition_comparison](https://github.com/thouis/partition-comparison) and split VI is calculated using [gala](https://github.com/janelia-flyem/gala).

In [1]:
%matplotlib inline
import mahotas as mh
import cPickle
from gala import evaluate as ev
import numpy as np
import partition_comparison
import matplotlib.pyplot as plt
import cv2

In [2]:
def disk(radius):
    y, x = np.ogrid[-radius:radius+1, -radius:radius+1]
    return x**2 + y**2 <= radius**2


def watershed(probs, radius):
    sel = disk(radius)
    minima = mh.regmin(probs, Bc=sel)
    markers, nr_markers = mh.label(minima)
    return mh.cwatershed(probs, markers)

def partition_VI(putative, gold):
    mask = gold != 0
    VI_res = partition_comparison.variation_of_information(gold[mask], putative[mask].astype(gold.dtype))
    return VI_res

def evaluate_VI(pred,ground_truth,radii):

    if radii.size >1:
        VI = np.zeros((radii.size,3))
        r = 0
        for radius in radii:
            # VI_metric -> undersegmentation error,oversegmentation error,VI

            segmentation = watershed(pred,radius)
            VI_split =  ev.split_vi(segmentation, ground_truth)
            VI[r,0] += VI_split[0]
            VI[r,1] += VI_split[1]

            VI_partition = partition_VI(segmentation, ground_truth)    
            VI[r,2] += VI_partition   

            r += 1
    else:
        try:
            segmentation = watershed(pred,radii[0])
            VI = partition_VI(segmentation, ground_truth) 
        except:
            segmentation = watershed(pred,radii)
            VI = partition_VI(segmentation, ground_truth)
    
    return VI

def remove_synapse(img, threshold = 399):  
    for m in xrange(img.shape[0]):
        for n in xrange(img.shape[1]):
            if img[m,n] > threshold:
                img[m,n] = 0
    return img

def find_min(VI):
    # Find minimum VI value
    VI_min_pos = np.argmin(VI[:,2])
    return VI[VI_min_pos], VI_min_pos
    

### Calculate VI/Error
VI truth is the estimated VI for the membrane edges that are used as ground truth and represent the loss going from segmentation to edges and back. 

In [ ]:
address1          = "../results/48x48_av/"
pred_48x48_av     = np.load(address1 + "/results/output.npy")
y1                = np.load(address1 + "/results/y.npy")
pred_window_size  = np.load(address1 + "/results/pred_window_size.npy")
gap1 = (pred_window_size[0]-pred_window_size[1])/2

address2          = "../results/48x48_2/"
pred_48x48         = np.load(address2 + "/results/output.npy")
y2                = np.load(address2 + "/results/y.npy")
pred_window_size  = np.load(address2 + "/results/pred_window_size.npy")
gap2 = (pred_window_size[0]-pred_window_size[1])/2

# Load in test-addresses
f = file(address1 + "/pre_processed/" + "test_adress.dat", 'rb')
test_address = cPickle.load(f)
f.close()


In [ ]:
radii = np.linspace(7, 85, 30)
VI_48x48_av = np.zeros((radii.size,3))
VI_48x48 = np.zeros((radii.size,3))

n = 0
for address_img in test_address:
    ground_truth = mh.imread("../" + address_img)
    ground_truth = ground_truth[gap2:-gap2,gap2:-gap2]
    ground_truth = ground_truth.astype(np.uint16)
    ground_truth = remove_synapse(ground_truth)
    VI_48x48     += evaluate_VI( pred_48x48[n+5,0],ground_truth,radii)
    
    ground_truth = mh.imread("../" + address_img)
    ground_truth = ground_truth[gap1:-gap1,gap1:-gap1]
    ground_truth = ground_truth.astype(np.uint16)   
    VI_48x48_av       += evaluate_VI(pred_48x48_av[n,0],ground_truth,radii)
    
    n += 1
    
# Average VI
VI_48x48_av    /= float(n) 
VI_48x48       /= float(n) 

print "Finshed VI/watershed"

# Calculate min VI
VI_min_48x48_av,min_index = find_min(VI_48x48_av)
VI_min_48x48,min_index = find_min(VI_48x48)

# Calculate pixel erro
pixel_error_48x48_av = np.mean(np.abs(pred_48x48_av-y1))
pixel_error_48x48 = np.mean(np.abs(pred_48x48-y2))

# Estimate Ideal VI
#radii_min = np.array([radii[min_index]],dtype=np.int32)

#VI_truth = evaluate_VI(y2[-1],ground_truth,radii_min)
#print "Estimated VI Truth:", VI_truth
    

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.title("Variation of Information")
plt.xlabel("Watershed radius (pixels)")
plt.ylabel("VI")
plt.plot(radii,VI_48x48_av[:,2],label="2x2 output window")
plt.plot(radii,VI_48x48[:,2],label="48x48 output window")
plt.legend(loc="best")

plt.subplot(1,2,2)
plt.title("Split VI")
plt.plot(radii,VI_48x48_av[:,0],label = "Underseg. error, 2x2",linestyle="--",color="blue")
plt.plot(radii,VI_48x48_av[:,1],label = "Overseg. error, 2x2",color="blue")
plt.plot(radii,VI_48x48[:,0],label = "Underseg. error, 48x48",linestyle="--",color="red")
plt.plot(radii,VI_48x48[:,1],label = "Overseg. error, 48x48",color="red")
plt.legend(loc="best")
plt.xlabel("Watershed radius (pixels)")
plt.ylabel("VI")

plt.savefig("comparison_2x48_VI")

### Best Segmentation Performance, 48x48 averaged

In [ ]:
print "Variation of Information (VI):",VI_min_48x48_av[2]
print "VI, undersegmentation error:", VI_min_48x48_av[0]
print "VI, oversegmentation error:", VI_min_48x48_av[1]
print "Pixel-wise error:", pixel_error_48x48_av

### Best Segmentation Performance, 48x48

In [ ]:
print "Variation of Information (VI):",VI_min_48x48[2]
print "VI, undersegmentation error:", VI_min_48x48[0]
print "VI, oversegmentation error:", VI_min_48x48[1]
print "Pixel-wise error:", pixel_error_48x48

### Example pictures

In [ ]:
#pred_2x2 = np.load(address1 + "/results/output.npy")
#pred_48x48 = np.load(address2 + "/results/output.npy")
y = np.load(address1 + "/results/y.npy")

for n in xrange(y.shape[0]):

    plt.figure(figsize=(12,4))
    ax1 = plt.subplot(1,3,1)
    ax1.set_title("Prediction 48x48 averaged")
    plt.imshow(pred_48x48_av[n,0],cmap=plt.cm.gray)
    ax1.grid(True)
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])

    ax2 = plt.subplot(1,3,2)
    ax2.set_title("Prediction 48x48")
    plt.imshow(pred_48x48[5+n,0],cmap=plt.cm.gray)
    ax2.grid(True)
    ax2.set_xticklabels([])
    ax2.set_yticklabels([])

    plt.subplot(1,3,3)
    ax3 = plt.subplot(1,3,3)
    ax3.set_title("Ground Truth")
    plt.imshow(y[n,0],cmap=plt.cm.gray)
    ax3.grid(True)
    ax3.set_xticklabels([])
    ax3.set_yticklabels([])
    
    plt.savefig("comparison_48x48_avg_" + str(n))